In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import glob
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus']=False # 负号

批量读取不同策略的收益情况, 进行改名并计算每日盈亏

In [20]:
def read_files(path):
    file_list = glob.glob(path + '/*.xlsx')
    all_data = []                             # 用于储存数据的列表
    for _ in file_list:
        data = pd.read_excel(_)
        all_data.append(data)

    return all_data


def batch_process(all_data):
    new_col_names = {'时间':'Date',
                     '基准收益':'refer_profit',
                     '策略收益':'strategy_profit',
                     '当日盈利':'gain', 
                     '当日亏损':'loss',
                     '当日买入':'buy',
                     '当日卖出':'sell',
                     '超额收益(%)':'excess'}
    for data in all_data:
        data.rename(columns=new_col_names, inplace=True)

    # 每日盈亏 = 盈利 + 亏损
    for data in all_data:
        data['profit'] = data['gain'] + data['loss']


def pos_fetch(all_data):
    refresh = []
    for data in all_data:
        data['日期'] = pd.to_datetime(data['日期'])
        data = data.groupby('日期')['仓位'].sum().reset_index()
        refresh.append(data)
    return refresh

In [ ]:
folder_path = r'C:\Users\20536\Desktop\rock\策略研究\小市值_聚宽'
data_list = read_files(folder_path)
batch_process(data_list)

保存并批量覆写原文件

In [ ]:
for file_path, data in zip(glob.glob(folder_path+'/*.xlsx'), data_list):
    data.to_excel(file_path, index=False)

提取不同策略的每日仓位比例

In [21]:
pos_path = r'C:\Users\20536\Desktop\rock\策略研究\小市值_聚宽\持仓'
pos_list = read_files(pos_path)
refresh = pos_fetch(pos_list)

In [22]:
for path, datas in zip(glob.glob(pos_path+'/*.xlsx'), refresh):
    datas.to_excel(path, index=False)